# Web スクレイピング入門

Python の使い方も学び、Web ページの構造も理解したので、
早速 Web スクレイピングをしていきましょう。

まず、このページでは手始めに [Yahoo news のトピックス一覧](https://news.yahoo.co.jp/topics) から、各記事のタイトルを収集することを目指します。

## requests

まず、python を使って、Web ページのコンテンツをとってみましょう。
これには、 `requests` というモジュールを使います。
python にも似たようなものがありますが、 `requests` の方が使いやすいです。
コンテンツが欲しい Web ページの url について、 `requests.get(url)` とすると、Web サーバーに Web ページのコンテンツを送ってくれという要求をすることができます。

In [1]:
import requests
url = "https://news.yahoo.co.jp/topics"
response = requests.get(url)
response.status_code

200

最後の行は、ステータスコードというものを表していてます。
ステータスコードは Web サーバーへの要求が上手くいったかを判別することができるので、毎回チェックしておきましょう。

200 は、Web サーバーの要求が無事成功したことを示しています。
ステータスコードは 200 以外にも、様々な値を返すので、やりながら学んでいけば良いでしょう。

## BeautifulSoup

次に、Web ページのコンテンツからデータを取りやすいようにします。
ここでは、`BeautifulSoup` を用います。`BeautifulSoup` は使い方がわかりやすいので、入門としては良いでしょう。

In [2]:
from bs4 import BeautifulSoup
bs = BeautifulSoup(response.content,"lxml")

`lxml` というのは、htmlのパーサーの一つです。BeautifulSoup には 4 つほどのパーサーを使えますが、lxml は使いやすさと早さを兼ね備えているので、これを使います。

HTML の章で解説しましたが、Web ページは様々な要素が入れ子になっています。
したがって、自分の欲しい情報をとるには、その情報が置いてある要素を上手く指定することが大事です。
要素を指定する方法は、CSS のところで解説しました。

それでは、我々の欲しい情報は一体どこに位置しているのでしょうか？
実は、Google Chrome や　Firefox などのモダンブラウザには、それを分かりやすくする機能が備わっています。
Web ページを開いて、`Ctrl + Shift + I` を押すか、右クリックを押してから、検証（インスペクタ）を押してみましょう。

画面が 2 分割され、左側に Web ページ、右側に html が表示されるでしょう。
右側の左上のカーソルのマークをクリックしてから、左側の適当なところを押してみましょう。

すると、そのテキストが html 内のどこにあるかが、右側に表示されます。この機能を使って、欲しい情報はどこにあるのかを探っていきます。ここでは、トピックス一覧の内容が欲しいので、その内のどれかを押してみましょう。

一覧の内容は全てリンクが張られているので、 `a` 要素に含まれていることがわかりました。

![a](image/inspect_a.png)

`BeautifulSoup` は `select` で CSS セレクターを使うことができます。
下の例では `a` 要素を取ってきています。
`select` は CSS セレクターで指定した要素をタグごとにリストに含めます。

In [3]:
bs.select('a')[0]

<a id="yjPagetop" name="yjPagetop"></a>

最初の要素しか表示していないのは、`a` 要素に当てはまるのが多すぎるからです！
Web ページは多くのところでリンクを貼っているので、`a` 要素だと制限が緩すぎます。

別の方法を考えましょう。各トピックの全体を検証してみると、トピック毎に `ul` 要素を使ってリストにしていることがわかります。それでは、`ul` 要素を指定してみましょう。

![ul](image/inspect_ul.png)

In [4]:
# 表示は省略
# bs.select('ul')

先程よりはましですが、まだ余計なものが残っています。
よく見ると、この `ul` 要素には `fl` というクラス名が割り当てられています。また、右に位置するトピックスの `ul` 要素には `fr` というクラス名が割り当てられています。この 2 つのクラス名を指定してみましょう。
`.fl, .fr` とすることで複数の要素を指定することができます。

In [5]:
topics = bs.select('.fl, .fr')
topics[0]

<ul class="fl">
<li class="ttl"><a href="https://news.yahoo.co.jp/hl?c=dom" onmousedown="this.href='https://news.yahoo.co.jp/hl?c=dom'">国内</a></li>
<li><a href="https://news.yahoo.co.jp/pickup/6253447" onmousedown="this.href='https://news.yahoo.co.jp/pickup/6253447'">防衛相 北のEMP兵器に否定的<span class="icPhoto">写真</span><span class="icNew">new</span></a></li>
<li><a href="https://news.yahoo.co.jp/pickup/6253442" onmousedown="this.href='https://news.yahoo.co.jp/pickup/6253442'">秋田県で震度4 津波心配なし<span class="icPhoto">写真</span><span class="icNew">new</span></a></li>
<li><a href="https://news.yahoo.co.jp/pickup/6253427" onmousedown="this.href='https://news.yahoo.co.jp/pickup/6253427'">日米 北への圧力強化を最優先<span class="icPhoto">写真</span></a></li>
<li><a href="https://news.yahoo.co.jp/pickup/6253434" onmousedown="this.href='https://news.yahoo.co.jp/pickup/6253434'">子の引き渡し 拒否なら制裁金<span class="icPhoto">写真</span></a></li>
<li><a href="https://news.yahoo.co.jp/pickup/6253423" onmousedown="this.href='https://news.

上手くいきました！各トピックがリストの各要素に入ってます。
それでは、ここからトピック毎に記事名を格納していきましょう。
各 `li` 要素には、トピックのタイトル、記事名が入っています。 

リストの各要素をさらに `li` 要素毎に分けます。要素ごとに分けたら、`text` メソッドを使って、要素の内容を取り出します。

In [6]:
topics[0].select('li')[0].text

'国内'

この作業をループ化して、`dict = {トピックのタイトル:[記事A,記事B]}` というように、辞書型にしていきます。
`li` 要素の最初の要素はトピックのタイトルなので、これをキーにします。
また、最後の要素は記事のタイトルではないので、これは含みません。

In [7]:
news_topics = {}
for news in topics:
    topic = news.select('li')[0].text
    news_topics[topic] = [news_topic.text for news_topic in news.select('li')[1:-2]]

In [8]:
news_topics['国内']

['防衛相 北のEMP兵器に否定的写真new',
 '秋田県で震度4 津波心配なし写真new',
 '日米 北への圧力強化を最優先写真',
 '子の引き渡し 拒否なら制裁金写真',
 '残業規制・高プロ 法案一本化写真',
 '民進の5人 来週にも離党へ写真',
 '老後奪われる?祖父母の孫疲れ写真new']

`[news_topic.text for news_topic in news.select('li')[1:-2]]` はリスト内包表記というものです。

ここでは、`li` 要素の 1 番目から最後の 1 つ前までの要素、`news.select('li')[1:-2]` を順に `news_topic` 格納し、
その要素の内容を `news_topic.text` で取り出し、リストに入れていく処理をしています。

ちなみに、この処理は次と同じです。
```
news_list = []
for news_topic in news.select('li')[1:-2]:
    news_list.append(news_topic.text)
```

リスト内包表記は、普通にループ文を書くよりもスッキリとして書け、また早く処理することができます。

せっかくなので、pandas の DataFrame に変換しましょう。
dict から DataFrame にするには、`from_dict` を使います。

また、写真、動画、new という文字は不要なので、pandas の `str.replace` を使って除去します。
`str` は DataFrame ではなく、Series のメソッドなので、`apply` を使って各列に適応させていきます。

In [9]:
import pandas as pd
topics_dt = pd.DataFrame.from_dict(news_topics)
topics_dt.apply( lambda x: x.str.replace(r'(new|写真|動画)',''))

,IT,エンタメ,スポーツ,国内,国際,地域,科学,経済
0,電子書籍読み感想文 なぜNG,with Bダイキ 父に今も怒られ,ダル MLB史最速1000奪三振,防衛相 北のEMP兵器に否定的,メキシコ死者61人 まるで爆撃,威嚇発砲され逃走の男発見か,山形城 特殊な「土塀」初発見,民泊など シェアエコ課税強化
1,モンストに480円の月額コース,27時間TV 生放送しない理由,イチロー 通算2500単打に到達,秋田県で震度4 津波心配なし,北が「核強国」強調 米批判も,屋内「禁煙」都が条例制定へ,阿修羅像 元は穏やかな表情?,北情勢緊迫 日銀打つ手なし
2,DMMもシェアサイクル参入へ,竹内涼真 視聴率王子と称賛,巨人ギャレットが帰国へ,日米 北への圧力強化を最優先,微量の放射性物質を検出 韓国,青森県警の警視 女性触り懲戒,地磁気の乱れ ピーク過ぎる,ツアー料金弁済 返金を増額へ
3,SNS人気家 実は存在せず,元SMAP3人今後は? 新仕事も,鳥谷2000安打 大不振乗り越え,子の引き渡し 拒否なら制裁金,米空母が横須賀出港 北けん制,ドンキで窃盗 約900万円被害,太陽フレア 北海道オーロラも,試験中断のMRJ 米で飛行再開
4,足が臭いと気絶 犬型ロボ開発,退所3人と中居 1年後に合流か,ガンバ新監督 セレーゾ氏浮上,残業規制・高プロ 法案一本化,最強ハリケーン週末に米直撃,だまされたふり逆手の新手口,オプジーボ 胃がんにも拡大,プジョー 罰金6500億円か
5,1.4億人のデータ流出か 米国,退所3人 ファンクラブ発足へ,女子バレー 強豪ブラジル撃破,民進の5人 来週にも離党へ,ネズミ襲撃で225カ所けが 仏,新東名でバス火災 乗客避難,「白頭山」調査 停滞の恐れ,カードローン残高 毎月公表へ
6,新iPhone 生産1カ月遅れか,レディー・ガガ 活動休止表明,3横綱休場 懸賞200本取り下げ,老後奪われる?祖父母の孫疲れ,相次ぐ米艦衝突 訓練不足横行,鈴木貴子議員 第1子を出産,リカオン くしゃみで投票?,神戸-沖縄結ぶフェリー運休へ
